In [1]:
import os
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"
os.environ["VLLM_USE_FLASHINFER"] = "0"
os.environ["VLLM_ATTENTION_BACKEND"] = "TRITON_ATTN"

In [2]:
!env | grep PYTORCH_CUDA_ALLOC_CONF
!env | grep VLLM_USE_FLASHINFER

VLLM_USE_FLASHINFER=0


In [3]:
!pip install -U numpy==1.26.4 scikit-learn==1.5.2 --no-cache-dir
!pip install transformers==4.56.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 238.3 MB/s eta 0:00:000:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 100.8 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.

In [4]:
!pip install -q --upgrade pip
!pip install -q vllm==0.11.1 
!pip install -q huggingface_hub
!pip install flask pyngrok flask_cors
!pip install -q bitsandbytes accelerate
# print("--- [FIX] Bắt đầu sửa lỗi xung đột numpy/sklearn ---")
# # # Buộc cài đặt lại các gói bị ảnh hưởng để đảm bảo chúng tương thích
# !pip install --upgrade --force-reinstall scikit-learn
# 
# print("--- [FIX] Đã hoàn tất sửa lỗi ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:0000:010:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cudf-cu12 25.2.2 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0

In [5]:
!rm -rf /kaggle/working/*

In [6]:
import subprocess
import time
import requests
import re
from pyngrok import ngrok
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from huggingface_hub import snapshot_download
import json
import os
import requests
import random
from openai import OpenAI
from typing import List, Set, Dict, Tuple, Optional

In [7]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")
if hf_token is None:
    raise RuntimeError("Thiếu HF_TOKEN. Vui lòng thêm vào Kaggle Secrets.")

In [8]:
login(token=hf_token)
print("✅ Đăng nhập Hugging Face thành công!")
# Ô 2b: Tải model safetensors từ repo gated


# MODEL_REPO = "ktam204/Qwen3-32B-AWQ-r16-lora-all-Pentest-swift"
# MODEL_REPO = "Qwen/Qwen3-32B-AWQ"
# LOCAL_DIR   = "/kaggle/tmp/Qwen3-32B"

# MODEL_REPO = "cpatonn/Qwen3-Coder-30B-A3B-Instruct-AWQ-4bit"
MODEL_REPO = "Qwen/Qwen2.5-32B-Instruct-AWQ"
# MODEL_REPO = "Qwen/Qwen3-4B-Instruct-2507"
LOCAL_DIR = "/kaggle/tmp/Qwen2.5-32B-Instruct-AWQ"


snapshot_download(
    repo_id=MODEL_REPO,
    local_dir=LOCAL_DIR,
    local_dir_use_symlinks=False
)
print("✅ Đã download xong vào:", LOCAL_DIR)

✅ Đăng nhập Hugging Face thành công!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

✅ Đã download xong vào: /kaggle/tmp/Qwen2.5-32B-Instruct-AWQ


In [9]:
!pip cache purge

Files removed: 450 (4819.4 MB)


In [10]:
# !ls $LOCAL_DIR

In [11]:
# !cat /usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py

In [12]:
# Log file names
vllm_logfile = 'vllm.log'

# 1) Launch vLLM server in background
# vllm_cmd = [
#     "python3", "-m", "vllm.entrypoints.openai.api_server",
#     "--model", LOCAL_DIR,
#     "--served-model-name", "Qwen3-32B",
#     "--dtype", "float16",             # ← use float16 on L4
#     "--gpu-memory-utilization", "0.9",
#     "--tensor-parallel-size", "4",
#     "--enforce-eager",
#     "--max-model-len", "16384",
#     # "--max-num-seqs", "1024",
#     #"--enable-reasoning",
#     # "--reasoning-parser", "deepseek_r1",
#     "--enable-auto-tool-choice", 
#     "--tool-call-parser", "hermes",
#     "--host", "0.0.0.0",
#     "--port", "8000",
# ]

vllm_cmd = [
    "python3", "-m", "vllm.entrypoints.openai.api_server",
    "--model", LOCAL_DIR,#"/kaggle/input/qwen-3/transformers/32b/1",
    "--served-model-name", "Qwen2.5-32B-Instruct-AWQ",
    "--dtype", "float16",             # ← use float16 on T4
    "--gpu-memory-utilization", "0.9",
    "--tensor-parallel-size", "2",
    "--enforce-eager",
    "--max-model-len", "16384",
    # "--reasoning-parser", "deepseek_r1",
    # "--enable-auto-tool-choice", 
    # "--tool-call-parser", "hermes",
    "--host", "0.0.0.0",
    "--port", "8000",
]

with open(vllm_logfile, "w") as logf:
    proc = subprocess.Popen(vllm_cmd, stdout=logf, stderr=subprocess.STDOUT)

# 2) Poll /v1/models up to 360 times, waiting 10s between tries (max 1 hour)
for attempt in range(360):
    try:
        resp = requests.get("http://127.0.0.1:8000/v1/models", timeout=10)
        if resp.status_code == 200:
            print(f"✅ vLLM is up on attempt {attempt+1}!")
            break
    except requests.RequestException as e:
       print(f"Error on attempt {attempt+1}: {e}")
       with open(vllm_logfile, "r") as f:
           lines = f.readlines()
       print("📄 Last 20 lines of log:")
       print("".join(lines[-50:]))
    print(f"Waiting for vLLM startup... ({attempt+1}/360)")
    time.sleep(10)
else:
    print("❌ Still not up after 1 hour. Check", vllm_logfile)
    proc.terminate()
    raise RuntimeError("vLLM startup timeout")

Error on attempt 1: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79a2404cd190>: Failed to establish a new connection: [Errno 111] Connection refused'))
📄 Last 20 lines of log:

Waiting for vLLM startup... (1/360)
Error on attempt 2: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79a2404dd450>: Failed to establish a new connection: [Errno 111] Connection refused'))
📄 Last 20 lines of log:

Waiting for vLLM startup... (2/360)
Error on attempt 3: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79a25b7eb310>: Failed to establish a new connection: [Errno 111] Connection refused'))
📄 Last 20 lines of log:
2025-11-28 08:03:10.113769

In [13]:
secrets = UserSecretsClient()
ngrok.set_auth_token(secrets.get_secret("ngrok_key"))

# 3) Open an HTTP tunnel on port 8000
tunnel = ngrok.connect(8000, "http")
public_url = tunnel.public_url
# 4) Print the public URL
print("🚀 Public URL for your vLLM API:", public_url)

# inspector_tunnel = ngrok.connect(4040, "http")
# inspector_url = inspector_tunnel.public_url
# print("🔍 Ngrok Inspector URL (logs):", inspector_url)

🚀 Public URL for your vLLM API: https://suboral-gertha-autoicous.ngrok-free.dev                    


In [14]:
 # !curl http://localhost:8000/v1/models

In [15]:
openAI_key = secrets.get_secret("OpenAI_key")
api_base = public_url.rstrip("/") + "/v1" 
client = OpenAI(base_url=api_base, api_key=openAI_key)
# Lấy model_id từ server vLLM
models = requests.get(f"{api_base}/models").json()["data"]
model_id = models[0]["id"]
print("🚀 Public URL for your vLLM API:", api_base)
print(model_id)

🚀 Public URL for your vLLM API: https://suboral-gertha-autoicous.ngrok-free.dev/v1
Qwen2.5-32B-Instruct-AWQ


In [16]:
PERTURBATION_JSON_FILE = "/kaggle/input/pertubation-feature/final_perturbation_p.json" # File input
OUTPUT_DIR = f"perturbation_code_snippets_by_{model_id}" # Thư mục output
# MODEL_NAME = "gemini-2.5-pro"

In [17]:
COMMON_PACKAGE_PREFIXES = [
    "com.google.android.gms.internal", "com.facebook.ads.internal", 
    "com.unity3d.player.utils", "org.apache.commons.io.file", 
    "com.applovin.sdk.core", "io.app.data.utils", 
    "util.common.lib", "com.app.services.core"
]

In [18]:
def _generate_natural_classname(feature_name: str, base_type: str) -> str:
    """Tạo tên class tự nhiên để tránh bị nghi ngờ."""
    prefix = random.choice(COMMON_PACKAGE_PREFIXES)
    name_part = feature_name.split('.')[-1]
    clean_name = "".join(c for c in name_part if c.isalnum())
    return f"{prefix}.{clean_name}{base_type}"

# --- [MỚI] HÀM TẠO PROMPT NÂNG CAO (R1 & R3) ---
def _generate_component_prompt(component_name: str, base_class: str, associated_feature: str = None) -> str:
    """
    Tạo prompt yêu cầu mã Smali AN TOÀN với kỹ thuật Opaque Predicate và Try-Catch.
    """
    
    # 1. Template XML (Intent Filter) - Giữ nguyên
    intent_xml = ""
    if associated_feature:
        if associated_feature.startswith("action."):
            action = associated_feature.split('.', 1)[1]
            intent_xml = f'<intent-filter><action android:name="{action}" /></intent-filter>'
        elif associated_feature.startswith("category."):
            cat = associated_feature.split('.', 1)[1]
            intent_xml = f'<intent-filter><category android:name="{cat}" /></intent-filter>'

    # 2. Định nghĩa Base Class
    base_class = "android.app.Service" # Default
    if comp_type == "Activity": base_class = "android.app.Activity"
    elif comp_type == "Receiver": base_class = "android.content.BroadcastReceiver"
    elif comp_type == "ContentProvider": base_class = "android.content.ContentProvider"

    # 3. Prompt Nâng cao: Yêu cầu Opaque Predicate & Try-Catch
    prompt = f"""
Bạn là chuyên gia bảo mật Android. Nhiệm vụ: Tạo mã Smali "Dead Code" để đánh lừa bộ phân tích tĩnh (Static Analysis) nhưng TUYỆT ĐỐI AN TOÀN khi chạy (Runtime).

Sử dụng kỹ thuật: 
1. **Opaque Predicate [42]**: Tạo điều kiện giả `if (1 == 0)` để mã bên trong không bao giờ chạy.
2. **Try-Catch Trap [29]**: Bao bọc mọi logic trong khối try-catch để nuốt lỗi.

HÃY TẠO 2 PHẦN:

---XML---
[Mã XML khai báo component trong AndroidManifest.xml]

---SMALI---
[Mã Smali hoàn chỉnh cho class]

YÊU CẦU CHI TIẾT:

1. XML:
   - Tên: {component_name}
   - Loại: {comp_type}
   - Intent-Filter: {intent_xml}
   - CHÚ Ý: Với Service/Receiver, thêm thuộc tính `android:enabled="true"` và `android:exported="false"`.

2. SMALI (QUAN TRỌNG):
   - Class: {component_name}
   - Super: {base_class}
   
   - **Cấu trúc phương thức:**
     Phải cài đặt ĐẦY ĐỦ các phương thức abstract để tránh Crash (VerifyError).
     
     * Nếu là **ContentProvider**, bắt buộc có: 
       onCreate()Z (return true), query, insert, update, delete, getType (return null/0).
     * Nếu là **Activity**, bắt buộc có:
       onCreate(Landroid/os/Bundle;)V -> PHẢI GỌI `invoke-super ... onCreate`.
     * Nếu là **Service**, bắt buộc có:
       onBind, onStartCommand.

   - **Chèn Nhiễu loạn (Perturbation Injection):**
     Trong một phương thức bất kỳ (ví dụ onCreate hoặc onStartCommand), hãy chèn đoạn mã Smali sau (Opaque Predicate):
     
     ```smali
     const/4 v0, 0x1
     const/4 v1, 0x0
     if-eq v0, v1, :cond_safe  # 1 != 0, luôn nhảy đến nhãn an toàn
     
     # --- VÙNG MÃ CHẾT (DEAD CODE) ---
     # Bộ phân tích tĩnh sẽ thấy code này, nhưng Runtime KHÔNG BAO GIỜ chạy vào đây.
     :try_start_0
     # (Tại đây LLM hãy tự chèn 1-2 API call vô hại hoặc liên quan đến feature "{associated_feature}" nếu có thể)
     # Ví dụ: Landroid/util/Log;->d(...)
     :try_end_0
     .catch Ljava/lang/Exception; {{:try_start_0 .. :try_end_0}} :catch_0
     
     :catch_0
     nop
     # -------------------------------
     
     :cond_safe
     # Tiếp tục luồng chính an toàn
     ```

   - **Quy tắc cú pháp:**
     - KHÔNG dùng tên tham số p1, p2 trong dòng .method.
     - Đảm bảo .locals đủ dùng (ít nhất .locals 2).
     - Return giá trị hợp lệ (0, null, void) ở cuối hàm.

VIẾT CODE CHÍNH XÁC, KHÔNG GIẢI THÍCH.
"""
    return prompt

In [19]:
def _generate_unique_classname(feature_name: str, base_type: str) -> str:
    """
    Tạo tên class Java duy nhất từ tên đặc trưng.
    Ví dụ: "action.BOOT_COMPLETED" -> "com.perturb.receiver.BootCompletedReceiver"
    """
    # Lấy phần chính của đặc trưng (ví dụ: BOOT_COMPLETED)
    name_part = feature_name.split('.')[-1]
    
    # Viết hoa các chữ cái đầu (BOOT_COMPLETED -> BootCompleted)
    class_name_suffix = "".join(part.capitalize() for part in name_part.split('_'))
    
    # Tạo tên class đầy đủ
    package_base = "com.perturb"
    class_name = f"{package_base}.{base_type.lower()}.{class_name_suffix}{base_type.capitalize()}"
    
    return class_name

# --- [MỚI] HÀM TRỢ GIÚP TẠO PROMPT CHO COMPONENT ---
def _generate_component_prompt(component_name: str, base_class: str, associated_feature: str = None) -> str:
    """
    Tạo prompt nghiêm ngặt yêu cầu cả XML và Smali cho một component.
    associated_feature: (Tùy chọn) action hoặc category cần thêm vào intent-filter.
    """
    
    intent_filter_xml = ""
    if associated_feature:
        if associated_feature.startswith("action."):
            action_name = associated_feature.split('.', 1)[1]
            intent_filter_xml = f"""
    <intent-filter>
        <action android:name="{action_name}" />
    </intent-filter>
"""
        elif associated_feature.startswith("category."):
            category_name = associated_feature.split('.', 1)[1]
            intent_filter_xml = f"""
    <intent-filter>
        <category android:name="{category_name}" />
    </intent-filter>
"""

    prompt = f"""
RẤT QUAN TRỌNG: Chỉ trả lời bằng mã code. KHÔNG được có bất kỳ lời giải thích hay bình luận nào.
Phản hồi của bạn BẮT BUỘC phải theo định dạng sau, bao gồm cả các dấu phân tách:

---XML---
[Mã XML để khai báo component]

---SMALI---
[Mã Smali cho class]

YÊU CẦU:
1.  (XML) Đoạn mã XML để khai báo component sau trong AndroidManifest.xml:
    {component_name}
    
    {f"Component này PHẢI chứa một <intent-filter> như sau: {intent_filter_xml}" if intent_filter_xml else ""}

2.  (Smali) Mã Smali tối giản cho class "{component_name}" kế thừa từ "{base_class}".
    Class Smali phải có một constructor rỗng (.method public constructor <init>()V) và các phương thức trừu tượng tối thiểu cần thiết để biên dịch (ví dụ: onBind, onCreate).
"""
    return prompt

In [20]:
# def generate_prompt(feature_name: str) -> Tuple[str, str, str]:
#     """
#     Tạo prompt cho LLM dựa trên tên đặc trưng.
#     Trả về: (prompt, file_type, suggested_filename)
#     """
    
#     # Định dạng CSV: permission.A, action.B, category.C, DService
    
#     if feature_name.startswith("permission."):
#         permission = feature_name.split('.', 1)[1]
#         prompt = f"""
# Hãy viết một đoạn mã XML duy nhất cho file AndroidManifest.xml để thêm quyền (uses-permission) sau:
# {permission}

# Chỉ cung cấp duy nhất đoạn mã XML, không giải thích gì thêm.
# """
#         # file_type="xml_permission", filename=tên đặc trưng
#         return (prompt, "xml_permission", f"{feature_name}.xml")

#     elif feature_name.startswith("action."):
#         base_type = "Receiver"
#         base_class = "android.content.BroadcastReceiver"
#         # Tạo tên class duy nhất, ví dụ: com.perturb.receiver.BootCompletedReceiver
#         component_name = _generate_unique_classname(feature_name, base_type)
#         # Tạo prompt yêu cầu cả XML (với action) và Smali
#         prompt = _generate_component_prompt(component_name, base_class, associated_feature=feature_name)
#         # file_type="component", filename=tên class
#         return (prompt, "component", component_name) 

#     elif feature_name.startswith("category."):
#         base_type = "Activity"
#         base_class = "android.app.Activity"
#          # Tạo tên class duy nhất, ví dụ: com.perturb.activity.LauncherActivity
#         component_name = _generate_unique_classname(feature_name, base_type)
#         # Tạo prompt yêu cầu cả XML (với category) và Smali
#         prompt = _generate_component_prompt(component_name, base_class, associated_feature=feature_name)
#          # file_type="component", filename=tên class
#         return (prompt, "component", component_name)

#     else:
#         # Đây là một component đã có tên (ví dụ: AudioCacheService)
#         component_name = feature_name
        
#         # Xác định loại component (heuristic)
#         comp_type = "Service" # Mặc định
#         if "Activity" in component_name:
#             comp_type = "Activity"
#         elif "Receiver" in component_name:
#             comp_type = "Receiver"
#         elif "Provider" in component_name:
#             comp_type = "ContentProvider"
            
#         base_class = {
#             "Service": "android.app.Service",
#             "Activity": "android.app.Activity",
#             "Receiver": "android.content.BroadcastReceiver",
#             "ContentProvider": "android.content.ContentProvider"
#         }.get(comp_type, "android.app.Service")

#         # Tạo prompt yêu cầu cả XML và Smali
#         prompt = _generate_component_prompt(component_name, base_class)
#         # file_type="component", filename=tên class
#         return (prompt, "component", component_name)


# --- [CẬP NHẬT] HÀM generate_prompt (SỬ DỤNG CÁC HÀM MỚI) ---
def generate_prompt(feature_name: str) -> Tuple[str, str, str]:
    """
    Tạo prompt cho LLM dựa trên tên đặc trưng. (Phiên bản nâng cao R3)
    Trả về: (prompt, file_type, suggested_filename)
    """
    
    # 1. Permissions (An toàn nhất - Giữ nguyên)
    if feature_name.startswith("permission."):
        permission = feature_name.split('.', 1)[1]
        prompt = f"""
Hãy viết một dòng XML duy nhất cho AndroidManifest.xml để thêm quyền:
{permission}
Định dạng: <uses-permission android:name="{permission}" />
Chỉ trả về mã XML.
"""
        return (prompt, "xml_permission", f"{feature_name}.xml")

    # 2. Actions & Categories & Components
    # Xác định loại component dựa trên tên hoặc heuristic
    comp_type = "Service" # Mặc định an toàn
    if "Activity" in feature_name or feature_name.startswith("category."):
        comp_type = "Activity"
    elif "Receiver" in feature_name or feature_name.startswith("action."):
        comp_type = "Receiver"
    elif "Provider" in feature_name:
        comp_type = "ContentProvider"
    
    # Tạo tên class tự nhiên
    class_name = _generate_natural_classname(feature_name, comp_type)
    
    # Tạo prompt mạnh mẽ
    prompt = _generate_component_prompt(class_name, comp_type, associated_feature=feature_name)
    
    return (prompt, "component", class_name)

In [21]:
def call_vllm_client(client, model_id, prompt, retries=3, delay=5):
    """
    Gửi prompt đến máy chủ vLLM qua API tương thích OpenAI.
    """
    # Tạo định dạng message theo chuẩn OpenAI
    messages = [
        {"role": "system", "content": "You are an expert Smali and Android XML code generator. You must follow all formatting instructions exactly."},
        {"role": "user", "content": prompt}
    ]
    
    for attempt in range(retries):
        try:
            print(f"  ... Đang gửi prompt (lần {attempt + 1})...")
            
            resp = client.chat.completions.create(
                model=model_id,
                messages=messages,
                max_tokens=2048,   # Tăng max_tokens để chứa đủ code
                temperature=0.1,   # Cần độ chính xác, không cần sáng tạo
                top_p=0.9,
            )
            
            code_snippet = resp.choices[0].message.content.strip()
            
            # Trích xuất code (giống logic cũ)
            if "```" in code_snippet:
                try:
                    code_snippet = code_snippet.split("```")[1]
                    if '\n' in code_snippet:
                        code_snippet = code_snippet.split('\n', 1)[1]
                except Exception:
                    # Nếu trích xuất lỗi, cứ trả về nội dung thô
                    pass
            return code_snippet.strip()
        
        except Exception as e:
            print(f"  LỖI khi gọi vLLM: {e}. Đang thử lại sau {delay} giây...")
            time.sleep(delay)
            
    print(f"  LỖI: Không thể lấy phản hồi từ vLLM sau {retries} lần thử.")
    return None

In [22]:
def save_snippet(output_dir, file_type, filename, content):
    """Lưu đoạn mã vào đúng thư mục."""
    
    # Đổi tên: xml_permission -> xml_manifest
    if file_type == "xml_permission":
        save_dir = os.path.join(output_dir, "xml_manifest_additions")
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, filename)
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"  ✓ Đã lưu XML (Permission): {save_path}")

    elif file_type == "component":
        # Tách XML và Smali từ phản hồi
        xml_content = ""
        smali_content = ""
        
        # Sử dụng regex để tìm kiếm linh hoạt hơn
        xml_match = re.search(r"---XML---(.*?)(?=---SMALI---)", content, re.DOTALL)
        smali_match = re.search(r"---SMALI---(.*)", content, re.DOTALL)

        if xml_match and smali_match:
            xml_content = xml_match.group(1).strip()
            smali_content = smali_match.group(1).strip()
            
            # Xóa các dấu ``` nếu LLM trả về
            xml_content = xml_content.replace("```xml", "").replace("```", "").strip()
            smali_content = smali_content.replace("```smali", "").replace("```", "").strip()
        else:
            print(f"  LỖI: Phản hồi cho component {filename} không đúng định dạng (thiếu '---XML---' hoặc '---SMALI---').")
            print("  --- NỘI DUNG PHẢN HỒI GÂY LỖI ---")
            print(content)
            print("  ---------------------------------")
            return 

        # Lưu file XML (Khai báo component)
        save_dir_xml = os.path.join(output_dir, "xml_manifest_additions")
        os.makedirs(save_dir_xml, exist_ok=True)
        # Tên file XML là tên class, ví dụ: com.perturb.receiver.BootCompletedReceiver.xml
        save_path_xml = os.path.join(save_dir_xml, f"{filename}.xml")
        with open(save_path_xml, 'w', encoding='utf-8') as f:
            f.write(xml_content)
        print(f"  ✓ Đã lưu XML (Component): {save_path_xml}")

        # Lưu file Smali (Mã code của component)
        save_dir_smali = os.path.join(output_dir, "smali_additions")
        # Tên file là tên class, ví dụ: com.perturb.receiver.BootCompletedReceiver
        # Chuyển thành đường dẫn: smali/com/perturb/receiver/BootCompletedReceiver.smali
        smali_path = os.path.join("smali", *filename.split('.')) + ".smali" 
        full_smali_path = os.path.join(save_dir_smali, smali_path)
        os.makedirs(os.path.dirname(full_smali_path), exist_ok=True)
        with open(full_smali_path, 'w', encoding='utf-8') as f:
            f.write(smali_content)
        print(f"  ✓ Đã lưu Smali: {full_smali_path}")

In [23]:
def main_code_generator():
    """Hàm chính điều khiển quá trình sinh mã."""
    
    print(f"--- Bắt đầu Bước 4a: Sinh mã Nhiễu loạn (dùng vLLM) ---")
    
    # 1. Tải các đặc trưng nhiễu loạn
    try:
        with open(PERTURBATION_JSON_FILE, 'r') as f:
            data = json.load(f)
        features = data.get("perturbation_features", [])
        if not features:
            print(f"LỖI: Không tìm thấy đặc trưng nào trong {PERTURBATION_JSON_FILE}")
            return
        print(f"Tìm thấy {len(features)} đặc trưng nhiễu loạn cần tạo mã.")
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file JSON nhiễu loạn tại: {PERTURBATION_JSON_FILE}")
        print("Vui lòng chạy Giai đoạn Fine-tune trước hoặc kiểm tra lại đường dẫn.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file JSON: {e}")
        return

    # 2. Cấu hình vLLM Client (SỬ DỤNG BIẾN TỪ NOTEBOOK)
    try:
        # Biến 'public_url' được tạo từ ô code số 5
        
        print(f"✓ Đã kết nối với vLLM Client tại: {api_base}")
        print(f"✓ Sử dụng model: {model_id}")
    except Exception as e:
        print(f"LỖI: Không thể kết nối vLLM. Đảm bảo server đang chạy.")
        print(f"Đã thử gọi URL: {public_url}, Lỗi: {e}")
        return

    # 3. Tạo thư mục output
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 4. Lặp qua từng đặc trưng và sinh mã
    print("\n--- Bắt đầu sinh mã ---")
    for feature in features:
        print(f"\nĐang xử lý đặc trưng: {feature}")
        
        # Tạo prompt
        prompt, file_type, filename = generate_prompt(feature)
        
        # Gọi vLLM
        code_content = call_vllm_client(client, model_id, prompt)
        
        # Lưu kết quả
        if code_content:
            save_snippet(OUTPUT_DIR, file_type, filename, code_content)
        else:
            print(f"  LỖI: Bỏ qua việc lưu file cho {feature} do không nhận được code.")
            
    print("\n--- Hoàn tất Bước 4a (vLLM) ---")
    print(f"Toàn bộ mã nhiễu loạn đã được lưu tại: {OUTPUT_DIR}")

In [24]:
if __name__ == "__main__":
    main_code_generator()

--- Bắt đầu Bước 4a: Sinh mã Nhiễu loạn (dùng vLLM) ---
Tìm thấy 33 đặc trưng nhiễu loạn cần tạo mã.
✓ Đã kết nối với vLLM Client tại: https://suboral-gertha-autoicous.ngrok-free.dev/v1
✓ Sử dụng model: Qwen2.5-32B-Instruct-AWQ

--- Bắt đầu sinh mã ---

Đang xử lý đặc trưng: AudioCacheService
  ... Đang gửi prompt (lần 1)...
  ✓ Đã lưu XML (Component): perturbation_code_snippets_by_Qwen2.5-32B-Instruct-AWQ/xml_manifest_additions/org.apache.commons.io.file.AudioCacheServiceService.xml
  ✓ Đã lưu Smali: perturbation_code_snippets_by_Qwen2.5-32B-Instruct-AWQ/smali_additions/smali/org/apache/commons/io/file/AudioCacheServiceService.smali

Đang xử lý đặc trưng: CleanerService
  ... Đang gửi prompt (lần 1)...
  ✓ Đã lưu XML (Component): perturbation_code_snippets_by_Qwen2.5-32B-Instruct-AWQ/xml_manifest_additions/com.google.android.gms.internal.CleanerServiceService.xml
  ✓ Đã lưu Smali: perturbation_code_snippets_by_Qwen2.5-32B-Instruct-AWQ/smali_additions/smali/com/google/android/gms/inter

In [25]:
import shutil
import os

folder_to_zip = OUTPUT_DIR
output_zip_file = "/kaggle/working/perturbation_code_snippets_3" # Tên file .zip (không cần .zip)

# Kiểm tra xem thư mục có tồn tại không
if os.path.isdir(folder_to_zip):
    try:
        # Tạo file zip
        shutil.make_archive(
            output_zip_file,  # Tên file zip output
            'zip',            # Định dạng nén
            folder_to_zip     # Thư mục cần nén
        )
        print(f"✅ Nén thành công! Đã tạo file: {output_zip_file}.zip")
    except Exception as e:
        print(f"Lỗi khi nén: {e}")
else:
    print(f"LỖI: Không tìm thấy thư mục: {folder_to_zip}")

✅ Nén thành công! Đã tạo file: /kaggle/working/perturbation_code_snippets_3.zip


In [26]:
# while True:
#     print("⚙️ Still running...")  # Hoặc bất kỳ dòng nào\
#     time.sleep(60)